In [75]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [76]:
flights = spark.read.csv('flights_small.csv', header=True)

FINS aquí els 3 passets necessaris per tenir un Spark DataFrame

In [77]:
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Ens han promès "estil" SQL, veiem-ho

In [78]:
flights.createOrReplaceTempView("flights")

In [79]:
query = 'FROM flights SELECT * LIMIT 10'

Cuidado, si no omplim el Catalog no funciona!

In [80]:
flights10 = spark.sql(query)

In [81]:
flights10.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Ara veurem com fer el camí de Spark DataFrame cap a Pandas Dataframe. Camí invers.

In [82]:
import pandas as pd

In [83]:
query2 = 'SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest'

In [84]:
flights_counts = spark.sql(query2)

Invoquem cap a Pandas

In [85]:
pd_counts = flights_counts.toPandas()

In [86]:
print(pd_counts.head())

  origin dest    N
0    SEA  RNO    8
1    SEA  DTW   98
2    SEA  CLE    2
3    SEA  LAX  450
4    PDX  SEA  144


Atenció: A partir d'aquí, imaginem que la tasca corre en un altre node

Tenim la vista "flights" al cataleg, anem a recuperar un Spark DataFrame que es podria dir flights sense problema però anomenarem flights2

In [87]:
flights2 = spark.table('flights')

In [88]:
flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Afegirem una columna nova amb la duració en hores del vol

In [89]:
flights2 = flights2.withColumn('duration_hours', flights2.air_time/60)

Comprovació que hi ha la nova columna

In [90]:
flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

Ara podem filtrar amb Filter() que és semblant a fer un WHERE de SQL

In [91]:
flights2.filter("air_time > 120").show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|2.1166666666666667|
|2014|    1| 15|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|  10|    37|2.0166666666666666|
|2014|    4| 19|    1236|  

El mateix filtre que el pas anterior amb una altra sintaxi, escull la que més t'agradi

In [92]:
flights2.filter(flights2.air_time > 120).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|2.1166666666666667|
|2014|    1| 15|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|  10|    37|2.0166666666666666|
|2014|    4| 19|    1236|  

Repetim en 2 passos.
Obtindrem els vols de llarga distància.

In [93]:
long_flights1 = flights2.filter("distance > 2720")
long_flights2 = flights2.filter(flights2.distance > 2720)
long_flights1.show()
long_flights2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|   duration_hours|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    3|  9|    2219|       -1|     644|      -11|     AA| N5CNAA|  1070|   SEA| MIA|     303|    2724|  22|    19|             5.05|
|2014|    1|  5|    2302|       57|     745|       65|     AA| N630AA|  1070|   SEA| MIA|     299|    2724|  23|     2|4.983333333333333|
|2014|    9|  6|    2336|        1|     811|       -5|     AA| N3JHAA|  1070|   SEA| MIA|     312|    2724|  23|    36|              5.2|
|2014|   11| 30|     105|        0|     929|      -11|     AA| N3APAA|   212|   SEA| MIA|     308|    2724|   1|     5|5.133333333333334|
|2014|    9| 21|    2327|       -8

Ens molesta veure tantes columnes en exemples anteriors? Cap problema, fem "selects"

In [94]:
# Primer grup de columnes
selected1 = flights2.select("tailnum","origin","dest")
# Segon grup de columnes (sintaxi alternativa)
temp = flights2.select(flights2.origin, flights2.dest, flights2.carrier)
# Primer Filtre
filterA = flights2.origin == "SEA"
# Segon Filtre
filterB = flights2.dest == "PDX"
# Encadenem filtres
selected2 = temp.filter(filterA).filter(filterB)
# Comprovacions
selected1.show(5)
selected2.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
+-------+------+----+
only showing top 5 rows

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 20 rows



Anem a fer una operació de transformació més elaborada que les anteriors

In [95]:
# Definim velicitat promig i li posem un ALIAS
avg_speed = (flights2.distance/(flights2.air_time/60)).alias("avg_speed")
speed1 = flights2.select("origin","dest","tailnum", avg_speed)
speed1.show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



La mateixa transformació anterior estil SQL

In [96]:
speed2 = flights2.selectExpr("origin","dest","tailnum","distance/(air_time/60) as avg_speed")
speed2.show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



Arriba el torn dels AGREGATS: EXERCICI PROPOSAT, analitza error i troba la solució per a cada origen busquem una cosa en concret


In [97]:
flights2 = flights2.withColumn("air_time", flights2.air_time.cast("integer"))
flights2.filter(flights2.origin == "SEA").groupBy().max('air_time').show()

+-------------+
|max(air_time)|
+-------------+
|          409|
+-------------+



In [98]:
flights2 = flights2.withColumn("distance", flights2.air_time.cast("integer"))
flights2.filter(flights2.origin == "PDX").groupBy().min('distance').show()

+-------------+
|min(distance)|
+-------------+
|           24|
+-------------+



In [99]:
flights2.filter(flights2.origin == "SEA").groupBy().avg('air_time').show()

+-----------------+
|    avg(air_time)|
+-----------------+
|160.4361496051259|
+-----------------+



Seguim fent agregats

In [100]:
by_plane = flights2.groupBy("tailnum")
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [101]:
by_origin = flights2.groupBy("origin")
by_origin.avg("air_time").show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



EXERCICI: prova de fer JOINS entre els DATAFRAMES flights i flights2


In [109]:
flights.join(flights2, on='tailnum', how='leftouter').show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|    duration_hours|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|2014|    2| 28|     700|        0|     854|       14|     VX|    84|   PDX| SFO|      88|      88|   7|     0|1.4666666666666666|
| N8